In [2]:
%matplotlib inline
from typing import Tuple

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from source import load_avenio_files, categorical_columns_to_lower
from transform import combine_tsv_files, load_process_and_store_spreadsheets, merge_mutation_spreadsheet_t0_with_t1, clean_mutation_columns

# First make a spreadsheet with merged t0 and t1.

In [3]:
spread_sheet_filename= "variant_list_20200409.xlsx"
spss_filename= "clinical_20200419.sav"
columns = [
    "Allele Fraction",
    "No. Mutant Molecules per mL",
    "CNV Score",
]
patient_mutations, phenotypes = load_avenio_files(
    spread_sheet_filename, spss_filename
)

In [13]:
patient_mutations.groupby(['Patient ID']).agg(np.sum)

,No Patient,ctDNA T0,ctDNA T1,PFS,PFS censor,OS,OS censor,No. Mutant Molecules per mL,Variant Depth,Unique Depth,CNV Score,Unnamed: 30,Unnamed: 31
Patient ID,,,,,,,,,,,,,
852,852.0,4,4,284,4,852,4,66.30,25.0,14482.0,0.000000,0.0,0.0
951,951.0,18,18,1242,18,1422,18,6642.85,4289.0,73893.0,26.825325,0.0,0.0
952,952.0,16,16,112,16,928,16,157890.00,5650.0,39763.0,93.196091,0.0,0.0
957,957.0,22,22,484,22,1034,22,130186.40,8999.0,205583.0,40.141279,0.0,0.0
962,962.0,6,6,10344,0,10344,0,453.70,191.0,13029.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7183,7183.0,12,12,1092,12,1908,12,597.40,318.0,77588.0,0.000000,0.0,0.0
7184,7184.0,0,0,721,0,721,0,0.00,0.0,0.0,0.000000,0.0,0.0
7185,7185.0,14,14,602,14,2016,14,595.50,295.0,88445.0,0.000000,0.0,0.0


In [4]:
def harmonic_mean(t0, t1):
    return t0 * t1 / (t0 + t1)

load_process_and_store_spreadsheets(
    transformation=harmonic_mean,
    all_filename_prefix="output/all__harmonic_mean_",
    train_filename_prefix="output/train__harmonic_mean_",
    test_filename_prefix="output/test__harmonic_mean_",
)

> /home/hylke/workspace/avenio/transform.py(274)load_process_and_store_spreadsheets()
    273     # Make a different document for each of the column pairs.
--> 274     for column in columns:
    275         column_pair = [f"T0: {column}", f"T1: {column}"]

ipdb> n
> /home/hylke/workspace/avenio/transform.py(275)load_process_and_store_spreadsheets()
    274     for column in columns:
--> 275         column_pair = [f"T0: {column}", f"T1: {column}"]
    276         # Filter out the column pairs, and remove empty fields.

ipdb> n
> /home/hylke/workspace/avenio/transform.py(277)load_process_and_store_spreadsheets()
    276         # Filter out the column pairs, and remove empty fields.
--> 277         mutations = spread_sheet[column_pair].dropna(how="all").fillna(0).reset_index()
    278         # Repair dirty cells (with per cent signs etc.).

ipdb> n
> /home/hylke/workspace/avenio/transform.py(279)load_process_and_store_spreadsheets()
    278         # Repair dirty cells (with per cent si

> /home/hylke/workspace/avenio/transform.py(283)load_process_and_store_spreadsheets()
    282         mutations_transposed = transform_column_pair(
--> 283             clean_mutation_sheet,
    284             # Vocabulary is the entire dataset.

ipdb> n
> /home/hylke/workspace/avenio/transform.py(285)load_process_and_store_spreadsheets()
    284             # Vocabulary is the entire dataset.
--> 285             gene_vocabulary=clean_mutation_sheet["Gene"].unique(),
    286             column_pair=column_pair,

ipdb> 
> /home/hylke/workspace/avenio/transform.py(286)load_process_and_store_spreadsheets()
    285             gene_vocabulary=clean_mutation_sheet["Gene"].unique(),
--> 286             column_pair=column_pair,
    287             transformation=transformation,

ipdb> 
> /home/hylke/workspace/avenio/transform.py(287)load_process_and_store_spreadsheets()
    286             column_pair=column_pair,
--> 287             transformation=transformation,
    288         )

ipdb> 
> 

BdbQuit: 

In [ ]:
def difference(t0, t1):
    return t1 - t0

load_process_and_store_spreadsheets(
    transformation=difference,
    all_filename_prefix="output/all__difference_",
    train_filename_prefix="output/train__difference_",
    test_filename_prefix="output/test__difference_",
)

In [ ]:
def relative_difference(t0, t1):
    return (t1-t0)/t0

load_process_and_store_spreadsheets(
    transformation=relative_difference,
    all_filename_prefix="output/all__relative_difference_",
    train_filename_prefix="output/train__relative_difference_",
    test_filename_prefix="output/test__relative_difference_",
)

In [ ]:
def up_or_down(t0, t1):
    return np.sign(t1 - t0)


load_process_and_store_spreadsheets(
    transformation=up_or_down,
    all_filename_prefix="output/all__sign_",
    train_filename_prefix="output/train__sign_",
    test_filename_prefix="output/test__sign_",
)

In [ ]:
# Harmonic mean genomic variable.
X_hm_af, y_hm_af = combine_tsv_files(
    "output/all__harmonic_mean__Allele Fraction.tsv",
    "output/all__harmonic_mean__CNV Score.tsv",
)

X_hm_af.merge(
    y_hm_af, 
    left_index=True, 
    right_index=True
).to_excel('output/all__harmonic_mean__af_cnv.xlsx')

In [ ]:
# Harmonic mean genomic variable.
X_hm_molecules, y_hm_molecules = combine_tsv_files(
    "output/all__harmonic_mean__No. Mutant Molecules per mL.tsv",
    "output/all__harmonic_mean__CNV Score.tsv",
)

X_hm_molecules.merge(
    y_hm_molecules, 
    left_index=True, 
    right_index=True
).to_excel('output/all__harmonic_mean__molecules_cnv.xlsx')

In [ ]:
# Harmonic mean genomic variable.
X_hm_molecules, y_hm_molecules = combine_tsv_files(
    "output/all__difference__No. Mutant Molecules per mL.tsv",
    "output/all__difference__CNV Score.tsv",
)

X_hm_molecules.merge(
    y_hm_molecules, 
    left_index=True, 
    right_index=True
).to_excel('output/all__difference__molecules_cnv.xlsx')

In [ ]:
# Harmonic mean genomic variable.
X_hm_molecules, y_hm_molecules = combine_tsv_files(
    "output/all__relative_difference__No. Mutant Molecules per mL.tsv",
    "output/all__relative_difference__CNV Score.tsv",
)

X_hm_molecules.merge(
    y_hm_molecules, 
    left_index=True, 
    right_index=True
).to_excel('output/all__relative_difference__molecules_cnv.xlsx')